In [1]:
!pip install semantic-kernel==1.31.0

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from semantic_kernel import __version__
__version__

'1.31.0'

In [2]:
import os
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from dotenv import load_dotenv

load_dotenv()

#Initializing the kernel
kernel = Kernel()
selectedService = "azureopenai"
service_id = "default"

kernel.add_service(
    AzureChatCompletion(service_id=service_id, 
                        deployment_name=os.getenv("DEPLOYMENT_NAME"),
                        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                        api_key=os.getenv("AZURE_OPENAI_API_KEY"))
)

#### Using the existing Plugin - Joke

In [3]:
# Adding the plugin to the kernel
plugin = kernel.add_plugin(parent_directory="..\..\semantic-kernel\prompt_template_samples", plugin_name="FunPlugin")

In [4]:
from semantic_kernel.functions import KernelArguments

#Invoking the plugin Joke
joke_function = plugin["Joke"]

joke = await kernel.invoke(
    joke_function,
    KernelArguments(input="time travel to dinosaur age", style="super silly"),
)
print(joke)

I tried time traveling to the dinosaur age, but my GPS kept saying, “In 65 million years, make a U-turn.”


#### Inline Functions

In [5]:
## Summarize agent
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig

prompt="""{{$input}}
Summarize the above content."""

execution_settings=AzureChatPromptExecutionSettings(
    service_id="default",
    ai_model_id="gpt-4.1",
    max_tokens=1000,
    temperature=0.7
)

prompt_template_config = PromptTemplateConfig(
    template=prompt, 
    name="Summarize Agent",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True)
    ],
    execution_settings=execution_settings,
)

In [6]:
summarize_agent=kernel.add_function(
    function_name="summarizeFunc",
    plugin_name="summarizePlugin",
    prompt_template_config=prompt_template_config
)

In [7]:
input_text = """ 
It took five ill-fated conversations with Jean-Paul Sartre before the Vietnamese philosopher Trần Đức Thảo finally broke with French philosophy. Between November 1949 and January 1950, Thảo and Sartre recorded and transcribed their conversations on the relation between Marxism and the new philosophy of existentialism, with the intention of publishing them. Sartre hoped to prove that Marxism and existentialism – of which he was the primary representative – were consistent with one another: reconcilable projects. However, the two men’s exchange of views collapsed before completion, under a series of recriminations. Thảo remained bitter about this, later referring to an ‘insidious campaign’ among Sartre’s ‘disciples’ to paint him as responsible for the failure of this planned project. Today, the conversations are still lost.

For Thảo, their disagreement lay in the fact that Sartre did not recognise Marxism’s philosophical seriousness. For Sartre, traditional Marxism offered an attractive social and political programme but lacked a real or serious philosophical account of being and human nature. Sartre developed his ideas about existentialism out of an ambition to provide the foundations for a new Left-wing philosophy for the 20th century."""

summary = await kernel.invoke(summarize_agent, input=input_text)
print(summary)

Between November 1949 and January 1950, Vietnamese philosopher Trần Đức Thảo and French philosopher Jean-Paul Sartre held a series of conversations about the relationship between Marxism and existentialism, intending to publish their discussions. Sartre believed the two philosophies could be reconciled, while Thảo disagreed, feeling Sartre did not take Marxism’s philosophical depth seriously. The project collapsed amid mutual recriminations, and Thảo later accused Sartre’s followers of unfairly blaming him for its failure. The transcripts of their conversations remain lost. Sartre viewed existentialism as a way to provide a serious philosophical foundation for the Left, which he felt traditional Marxism lacked.


In [8]:
### Semantic Analysis

prompt="""
Classify the sentiment of the following text.

Return ONLY ONE WORD from this list:
Positive
Negative
Neutral

Do NOT explain.
Do NOT rephrase the text.
Do NOT add any extra words.

Text: 
{{$input_text}}

"""

execution_settings=AzureChatPromptExecutionSettings(
    service_id="default",
    ai_model_id="gpt-4.1",
    max_tokens=5,
)

get_class=PromptTemplateConfig(
    template=prompt,
    name="get_class",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input_text", description="Text to classify", is_required=True)
    ],
    execution_settings=execution_settings,
)



classification_agent = kernel.add_function(
    function_name="classifyFunc",
    plugin_name="classifyPlugin",
    prompt_template_config=get_class
)

In [9]:
text="The new update is okay, but it could be faster."
analysis = await kernel.invoke(classification_agent, input_text=text)
print(analysis)

Neutral


##### ChatPlugin

In [10]:
prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

execution_settings = AzureChatPromptExecutionSettings(
    service_id=service_id,
    max_tokens=2000, 
    temperature=0.7,
)

chat_prompt_template_config = PromptTemplateConfig(
    name="chat",
    template_format="semantic-kernel",
    template=prompt, 
    input_variables=[
        InputVariable(name="user_input", description="User Input", is_required=True),
        InputVariable(name="history", description="Conversation History", is_required=True, 
                      allow_dangerously_set_content=True),
    ],
    execution_settings=execution_settings
)

chat_function = kernel.add_function(
    function_name="chat", 
    plugin_name="chatPlugin",
    prompt_template_config=chat_prompt_template_config,
)

In [11]:
from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()
chat_history.add_system_message("You are an helpful assistant, in giving the book recommendations.")

arguments = KernelArguments(user_input="Hi I am looking for book suggestions", history=ChatHistory)

In [12]:
response = await kernel.invoke(chat_function, arguments=arguments)
print(response)

Great! I'd love to help. Can you tell me a bit more about your interests? For example, do you prefer fiction or non-fiction? Any favorite genres, topics, or authors?


In [13]:
chat_history.add_assistant_message(str(response))

In [14]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")

    # Process the user message and get an answer
    answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(answer))

In [15]:
await chat("I like fiction, specifically thrillers.")

User: I like fiction, specifically thrillers.
ChatBot: Excellent choice! Here are some highly recommended thriller novels you might enjoy:

1. **"The Girl with the Dragon Tattoo" by Stieg Larsson**  
A gripping mystery with a compelling protagonist, set in Sweden. It’s the first in the Millennium series.

2. **"Gone Girl" by Gillian Flynn**  
A dark, twisty psychological thriller about a marriage gone wrong and the secrets that unravel.

3. **"The Silent Patient" by Alex Michaelides**  
A psychological thriller centered around a woman who refuses to speak after being accused of murder.

4. **"The Woman in the Window" by A.J. Finn**  
A Hitchcock-inspired thriller about an agoraphobic woman who believes she witnessed a crime.

5. **"Before I Go to Sleep" by S.J. Watson**  
A suspenseful story about a woman with amnesia who tries to piece together the truth about her life.

Would you like recommendations for a particular subgenre, like legal thrillers or spy novels? Or do you want more b

In [16]:
print(chat_history)

<chat_history><message role="system"><text>You are an helpful assistant, in giving the book recommendations.</text></message><message role="assistant"><text>Great! I'd love to help. Can you tell me a bit more about your interests? For example, do you prefer fiction or non-fiction? Any favorite genres, topics, or authors?</text></message><message role="user"><text>I like fiction, specifically thrillers.</text></message><message role="assistant"><text>Excellent choice! Here are some highly recommended thriller novels you might enjoy:

1. **"The Girl with the Dragon Tattoo" by Stieg Larsson**  
A gripping mystery with a compelling protagonist, set in Sweden. It’s the first in the Millennium series.

2. **"Gone Girl" by Gillian Flynn**  
A dark, twisty psychological thriller about a marriage gone wrong and the secrets that unravel.

3. **"The Silent Patient" by Alex Michaelides**  
A psychological thriller centered around a woman who refuses to speak after being accused of murder.

4. **"T